In [7]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, Add, MaxPooling2D, Flatten, Dense, Softmax, Activation, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import Adam, Nadam, SGD
import numpy as np
import h5py

In [8]:
tf.__version__
tf.test.is_gpu_available()

True

In [9]:
def dataGenerator(start_index, stop_index):
    def generator():
        indexes = shuff[start_index : stop_index]
        for i in indexes:
            x = np.array(hdf5['Xs'][i])
            y = np.array(hdf5['Ys'][i])
            yield x, y
    return generator

In [10]:
hdf5 = h5py.File('./hdf5/kyu_dataset_cor.hdf5','r')
batch_size = 64
dataset_size = hdf5['Xs'].shape[0]
val_start = 0.8
val_stop = 1.0

train_steps = int(dataset_size * (val_start)/batch_size)
val_steps = int(dataset_size * (val_stop - val_start)/batch_size)
print(f"train_steps: {train_steps}, val_steps: {val_steps}")
shuff = np.arange(dataset_size)
np.random.shuffle(shuff)

data_gen = dataGenerator(start_index = 0, stop_index = int(dataset_size * val_start)) # 80% of the complete dataset
dataset = tf.data.Dataset.from_generator(data_gen, 
                                         output_types=(tf.dtypes.float32, tf.dtypes.int32),
                                         output_shapes=(tf.TensorShape((19,19,19)),tf.TensorShape((1))))
dataset = dataset.batch(batch_size, drop_remainder=True)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

data_gen_valid = dataGenerator(start_index=int(dataset_size * val_start), # 20% of the complete dataset
                               stop_index=int(dataset_size * val_stop))
dataset_valid = tf.data.Dataset.from_generator(data_gen_valid, 
                                         output_types=(tf.dtypes.float32, tf.dtypes.int32),
                                         output_shapes=(tf.TensorShape((19,19,19)),tf.TensorShape((1))))
dataset_valid = dataset_valid.batch(batch_size, drop_remainder=True)
dataset_valid = dataset_valid.prefetch(tf.data.AUTOTUNE)


train_steps: 169592, val_steps: 42398


In [11]:
# resnet50
def block1(x, filters, kernel_size=3, stride=1, conv_shortcut=False):
    bn_axis=3
    preact = Activation("relu")(x)
    if conv_shortcut:
        shortcut = Conv2D(4 * filters, 1, strides=stride)(preact)
    else:
        shortcut = x
    
    x = Conv2D(filters, 1, strides=1, use_bias=False)(preact)
    x = Activation("relu")(x)
    x = ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    x = Conv2D(
        filters,
        kernel_size,
        strides=stride,
        use_bias=False
    )(x)
    x = Activation("relu")(x)
    x = Conv2D(4 * filters, 1)(x)
    x = Add()([shortcut, x])
    return x

def ResNet50(include_top=True,
             input_tensor=None,
             input_shape=None,
             pooling=False):
    inputs = Input(shape = input_shape)
    x = block1(inputs, 256, conv_shortcut=True)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    
    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dense(361, activation='softmax', name='fc1000')(x)

    model = Model(inputs, x, name='resnet50')
    return model

model = ResNet50(include_top=True,
                 input_tensor=None,
                 input_shape=(19, 19, 19),
                 pooling=False)
opt = Nadam(learning_rate = 0.0001)
model.compile(optimizer = opt,
              loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 19, 19, 19)  0           []                               
                                ]                                                                 
                                                                                                  
 activation_87 (Activation)     (None, 19, 19, 19)   0           ['input_2[0][0]']                
                                                                                                  
 conv2d_89 (Conv2D)             (None, 19, 19, 256)  4864        ['activation_87[0][0]']          
                                                                                                  
 activation_88 (Activation)     (None, 19, 19, 256)  0           ['conv2d_89[0][0]']       

                                )                                                                 
                                                                                                  
 add_32 (Add)                   (None, 19, 19, 1024  0           ['add_31[0][0]',                 
                                )                                 'conv2d_100[0][0]']             
                                                                                                  
 activation_99 (Activation)     (None, 19, 19, 1024  0           ['add_32[0][0]']                 
                                )                                                                 
                                                                                                  
 conv2d_101 (Conv2D)            (None, 19, 19, 256)  262144      ['activation_99[0][0]']          
                                                                                                  
 activatio

                                )                                 'conv2d_112[0][0]']             
                                                                                                  
 activation_111 (Activation)    (None, 19, 19, 1024  0           ['add_36[0][0]']                 
                                )                                                                 
                                                                                                  
 conv2d_113 (Conv2D)            (None, 19, 19, 256)  262144      ['activation_111[0][0]']         
                                                                                                  
 activation_112 (Activation)    (None, 19, 19, 256)  0           ['conv2d_113[0][0]']             
                                                                                                  
 zero_padding2d_37 (ZeroPadding  (None, 21, 21, 256)  0          ['activation_112[0][0]']         
 2D)      

                                )                                                                 
                                                                                                  
 conv2d_125 (Conv2D)            (None, 19, 19, 256)  262144      ['activation_123[0][0]']         
                                                                                                  
 activation_124 (Activation)    (None, 19, 19, 256)  0           ['conv2d_125[0][0]']             
                                                                                                  
 zero_padding2d_41 (ZeroPadding  (None, 21, 21, 256)  0          ['activation_124[0][0]']         
 2D)                                                                                              
                                                                                                  
 conv2d_126 (Conv2D)            (None, 19, 19, 256)  589824      ['zero_padding2d_41[0][0]']      
          

                                                                                                  
 activation_136 (Activation)    (None, 19, 19, 256)  0           ['conv2d_137[0][0]']             
                                                                                                  
 zero_padding2d_45 (ZeroPadding  (None, 21, 21, 256)  0          ['activation_136[0][0]']         
 2D)                                                                                              
                                                                                                  
 conv2d_138 (Conv2D)            (None, 19, 19, 256)  589824      ['zero_padding2d_45[0][0]']      
                                                                                                  
 activation_137 (Activation)    (None, 19, 19, 256)  0           ['conv2d_138[0][0]']             
                                                                                                  
 conv2d_13

 zero_padding2d_49 (ZeroPadding  (None, 21, 21, 256)  0          ['activation_148[0][0]']         
 2D)                                                                                              
                                                                                                  
 conv2d_150 (Conv2D)            (None, 19, 19, 256)  589824      ['zero_padding2d_49[0][0]']      
                                                                                                  
 activation_149 (Activation)    (None, 19, 19, 256)  0           ['conv2d_150[0][0]']             
                                                                                                  
 conv2d_151 (Conv2D)            (None, 19, 19, 1024  263168      ['activation_149[0][0]']         
                                )                                                                 
                                                                                                  
 add_49 (A

 conv2d_162 (Conv2D)            (None, 19, 19, 256)  589824      ['zero_padding2d_53[0][0]']      
                                                                                                  
 activation_161 (Activation)    (None, 19, 19, 256)  0           ['conv2d_162[0][0]']             
                                                                                                  
 conv2d_163 (Conv2D)            (None, 19, 19, 1024  263168      ['activation_161[0][0]']         
                                )                                                                 
                                                                                                  
 add_53 (Add)                   (None, 19, 19, 1024  0           ['add_52[0][0]',                 
                                )                                 'conv2d_163[0][0]']             
                                                                                                  
 activatio

                                                                                                  
 conv2d_175 (Conv2D)            (None, 19, 19, 1024  263168      ['activation_173[0][0]']         
                                )                                                                 
                                                                                                  
 add_57 (Add)                   (None, 19, 19, 1024  0           ['add_56[0][0]',                 
                                )                                 'conv2d_175[0][0]']             
                                                                                                  
 avg_pool (GlobalAveragePooling  (None, 1024)        0           ['add_57[0][0]']                 
 2D)                                                                                              
                                                                                                  
 fc1000 (D

In [12]:
callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, 
                                patience=3, verbose=0, mode='min'
                                 , restore_best_weights=True)
callback2 = tf.keras.callbacks.ModelCheckpoint('./models/model_kyu_v2_b64_f256_l100_cor_{epoch:02d}_{val_accuracy:.4f}.h5', 
                                            monitor='val_accuracy', 
                                            verbose=0, save_best_only=False, save_weights_only=False, 
                                            mode='max', save_freq="epoch")

In [7]:
# model = load_model('./models/model_kyu_f256_l88_mir_02_0.0035.h5')

In [13]:
history = model.fit(
    dataset,
    epochs = 500,
#     steps_per_epoch = train_steps,
    validation_data = dataset_valid,
#     validation_steps = val_steps,
    callbacks = [callback1, callback2]
)

Epoch 1/500
     31/Unknown - 32s 303ms/step - loss: 5.9015 - accuracy: 0.0045

KeyboardInterrupt: 

In [8]:
# model.save('./models/model_kyu_v2_b64_f256_l100_cor_5.h5')